# Testing

Testing is extremely important. Without testing, you cannot be sure that your code is doing what you think. Testing is an integral part of software development, and should be done *while* you are writing code, not after the code has been written.

No doubt so far, you have been manually checking that your code does the right thing. Perhaps you are tunning your code over a particular input file and making sure that you get a correct-looking plot out at the end. This is a start but how can you be sure that there's not a subtle bug that means that the output is incorrect? And if there *is* a problem, how will you be able to work out exactly which line of code it causing it?

In order to be confident that our code it giving a correct output, a *test suite* is useful which provides a set of known inputs and checks that the code matches a set of known, expected outputs. To make it easier to locate where a bug is occuring, it's a good idea to make each individual test run over as small an amount of code as possible so that if *that* test fails, you know where to look for the problem. In Python this "small unit of code" is usually a function.

Let's get started by making sure that our `add_arrays` function matches the outputs we expect. As a reminder, this is what the file `arrays.py` looks like (though you will have a second function, `subtract_arrays` in yours):

In [1]:
%%writefile arrays.py

"""
This module contains functions for manipulating and combining Python lists.
"""

def add_arrays(x, y):
    """
    This function adds together each element of the two passed lists.

    Args:
        x (list): The first list to add
        y (list): The second list to add

    Returns:
        list: the pairwise sums of ``x`` and ``y``.

    Examples:
        >>> add_arrays([1, 4, 5], [4, 3, 5])
        [5, 7, 10]
    """
    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ + y_)

    return z

Overwriting arrays.py


Since the name of the module we want to test is `arrays`, let's make a file called `test_arrays.py` which contains the following:

In [2]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    if output == expect:
        print("OK")
    else:
        print("BROKEN")

test_add_arrays()

Overwriting test_arrays.py


This script defines a function called `test_add_arrays` which defines some known input (`a` and `b`) and a known, matching output (`expect`). It passes them to the function `add_arrays` and compares the output to `expected`. It will either print `OK` or `BROKEN` depending on whether it's working or not. Finally, we explicitly call the test function.

In [3]:
%run test_arrays.py

OK


### Exercise

Break the test by changing either `a`, `b` or `expected` and rerun the test script. Make sure that it prints `BROKEN` in this case. Change it back to a working state once you've done this.

## Asserting

... Change it to use an assert.

The method used here works and runs the code correctly but it doesn't give very good output. If we had five tests in our file and three of them were failing we'd see something like:

```
OK
BROKEN
OK
BROKEN
BROKEN
```

We'd then have to cross-check back to our code to see which tests the `BROKEN`s referred to.

To be able to automatically relate the output of the failing test to the place where your test failed, you can use an `assert` statement.

An `assert` statement is followed by something which is either truthy of falsy. If it is truthy then nothing happens but if it is falsy then an exception is raised:

In [4]:
assert 5 == 5

In [5]:
assert 5 == 6

AssertionError: 

We can then use this in the test function:

In [6]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

test_add_arrays()

Overwriting test_arrays.py


..now when we run we get nothing printed on success:

In [7]:
%run test_arrays.py

...but on a failure we get an error printed like:

```
Traceback (most recent call last):
  File "test_arrays.py", line 13, in <module>
    test_add_arrays()
  File "test_arrays.py", line 11, in test_add_arrays
    assert output == expect
AssertionError
```

which, if we had many test functions being run would tell us which one failed and on which line.

...fist failure, wouldn't continue...

## pytest

...We want to automate the runnng of these tests and there is a tool called `pytest` which does this.

Remove the call to `test_add_arrays()` on the last line of the file:

In [8]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


And run `pytest`:

In [9]:
!COLUMNS=60 venv/bin/pytest

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.1, py-1.8.0, pluggy-0.13.0
rootdir: /home/matt/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 1 item                                           

test_arrays.py .                                     [100%]

==================== 1 passed in 4.26s =====================


pytest will automatically...

...naming...

...If we break the test then we see...



...

In [10]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 999]  # Changed this to break the test
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


In [11]:
!COLUMNS=60 venv/bin/pytest

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.1, py-1.8.0, pluggy-0.13.0
rootdir: /home/matt/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 1 item                                           

test_arrays.py F                                     [100%]

========================= FAILURES =========================
_____________________ test_add_arrays ______________________

    def test_add_arrays():
        a = [1, 2, 3]
        b = [4, 5, 6]
        expect = [5, 7, 999]  # Changed this to break the test
    
        output = add_arrays(a, b)
    
>       assert output == expect
E       assert [5, 7, 9] == [5, 7, 999]
E         At index 2 diff: 9 != 999
E         Use -v to get the full diff

test_arrays.py:11: AssertionError
==================== 1 failed in 0.52s =====================


The output from this is better than we saw with the `assert`. It's printing the full context of the contents of the test function with the line where the `assert` is failing being marked with a `>`. It then gives an expanded explanation of why the assert failed. Before we just got `AssertionError` but now it printsout the contents of `output` and `expect` and tells us that at index 2 of the list it's finding a `9` where we told it to expect a `999`.

...Make sure to change it back so that the test passes...

In [12]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]  # Changed this back to 9
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


### Exercise

Write a test which tests your `subtract_arrays` function from the previous chapter. Make sure it passes with a correct input/output and correctly fails if you break it on purpose. [<small>answer</small>](answer_subtract_test.ipynb)

## Avoid repeating ourselves

..lets add a second test to check a different set of inputs and outputs...

In [13]:
%%writefile test_arrays.py

from arrays import add_arrays

def test_add_arrays1():
    a = [1, 2, 3]
    b = [4, 5, 6]
    expect = [5, 7, 9]
    
    output = add_arrays(a, b)
    
    assert output == expect

def test_add_arrays2():
    a = [-1, -5, -3]
    b = [-4, -3, 0]
    expect = [-5, -8, -3]
    
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


In [14]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.1, py-1.8.0, pluggy-0.13.0 -- /home/matt/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 2 items                                          

test_arrays.py::test_add_arrays1 PASSED              [ 50%]
test_arrays.py::test_add_arrays2 PASSED              [100%]

==================== 2 passed in 0.25s =====================


This will work well but we've had to repeat ourselves almost entirely. The only difference between the two functions is the inputs and outputs of the function under test. Usually in this case in a function you would take these things as arguments and we can do the same thing here.

The actual logic of the function is the following:

```python
def test_add_arrays(a, b, expect):
    output = add_arrays(a, b)
    assert output == expect
```

We then just need a way of passing the data we want to check into this function. For this, pytest provides a feature called *parametrisation*. We label our fucntion with a *decoration* which allows pytest to run it mutliple times with different data.

To use this feature we must import the `pytest` module and use the `pytest.mark.parametrize` decorator like the following:

In [15]:
%%writefile test_arrays.py

import pytest

from arrays import add_arrays

@pytest.mark.parametrize("a,b,expect", [
    ([1, 2, 3],    [4, 5, 6],   [5, 7, 9]),
    ([-1, -5, -3], [-4, -3, 0], [-5, -8, -3]),
])
def test_add_arrays(a, b, expect):
    output = add_arrays(a, b)
    
    assert output == expect

Overwriting test_arrays.py


The `parametrize` decorator takes two arguments:
1. a string containing the names of the variables you want to pass in
2. a list containing the values of the variables you want to pass in

In this case, the test will be run twice with the following values:
1. `a=[1, 2, 3]`, `b=[4, 5, 6]`, `expect=[5, 7, 9]`
2. `a=[-1, -5, -3]`, `b=[-4, -3, 0]`, `expect=[-5, -8, -3]`

In [16]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.1, py-1.8.0, pluggy-0.13.0 -- /home/matt/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 2 items                                          

test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [ 50%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [100%]

==================== 2 passed in 0.24s =====================


### Exercise

- Add some more parameters to the `test_add_arrays` function.
- Parametrise the `subtract_arrays` test function.

## Failing correctly

The interface is made up of the parameters a function expects and the values that is returns. If a user of a function knows these things then they are able to use it correctly. This is why we make sure to include this information in the docstring for all our functions.

The other thing that is aprt of the interface of a function is any exceptions that are raised by it. If you need a refresher on exceptionns and error handling in Python, take a look at [the chapter on it in the Intermediate Python course](https://milliams.gitlab.io/intermediate_python/05%20Exceptions.html).

To add explicit error handling to our function we need to do two things:
1. add in a conditional `raise` statement
2. document the fact that the function may raise something

Let's add these to `arrays.py`:

In [17]:
%%writefile arrays.py

"""
This module contains functions for manipulating and combining Python lists.
"""

def add_arrays(x, y):
    """
    This function adds together each element of the two passed lists.

    Args:
        x (list): The first list to add
        y (list): The second list to add

    Returns:
        list: the pairwise sums of ``x`` and ``y``.
    
    Raises:
        ValueError: If the length of the lists ``x`` and ``y`` are different.

    Examples:
        >>> add_arrays([1, 4, 5], [4, 3, 5])
        [5, 7, 10]
    """
    
    if len(x) != len(y):
        raise ValueError("Both arrays must have the same length.")
    
    z = []
    for x_, y_ in zip(x, y):
        z.append(x_ + y_)

    return z

Overwriting arrays.py


We can then test that the function correctly raises the exception when passes appropriate data.  Inside a pytest function we can require that a specific exception is raised by using [`pytest.raises`](https://docs.pytest.org/en/latest/reference.html#pytest-raises) in a `with` block. `pytest.raises` takes as an argument the type of an exception and if the block ends without that exception being rasie will fail the test.

It may seem strange that we're testing and *requiring* that the function raises an error but it's important that if we've told our users that the code will produce a certain error in specific circumstances that it does as we promise.

In our code we add a new test called `test_add_arrays_error` which does the check we require.

In [18]:
%%writefile test_arrays.py

import pytest

from arrays import add_arrays

@pytest.mark.parametrize("a,b,expect", [
    ([1, 2, 3],    [4, 5, 6],   [5, 7, 9]),
    ([-1, -5, -3], [-4, -3, 0], [-5, -8, -3]),
])
def test_add_arrays(a, b, expect):
    output = add_arrays(a, b)
    
    assert output == expect

def test_add_arrays_error():
    a = [1, 2, 3]
    b = [4, 5]
    with pytest.raises(ValueError):
        output = add_arrays(a, b)

Overwriting test_arrays.py


In [19]:
!COLUMNS=60 venv/bin/pytest -v

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.1, py-1.8.0, pluggy-0.13.0 -- /home/matt/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 3 items                                          

test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [ 33%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [ 66%]
test_arrays.py::test_add_arrays_error PASSED         [100%]

==================== 3 passed in 0.46s =====================


### Exercise

- Add a runtime test to the `subtract_arrays` function to check for unequal-length arguments.
- Add a test for the exception.

## Doctests

If you remember from when we were documenting out `add_arrays` function, we had a small section which gave the reader an example of how to sue the function:

```
Examples:
    >>> add_arrays([1, 4, 5], [4, 3, 5])
    [5, 7, 10]
```

Since this is valid Python code, we can ask pytest to run this code and check that the output we claimed would be returned is correct. If we pass `--doctest-modules` to the `pytest` command, it will search `.py` files for docstrings with example blocks and run them:

In [20]:
!COLUMNS=60 venv/bin/pytest -v --doctest-modules

=================== test session starts ====================
platform linux -- Python 3.7.3, pytest-5.2.1, py-1.8.0, pluggy-0.13.0 -- /home/matt/courses/software_engineering_best_practices/venv/bin/python3
cachedir: .pytest_cache
rootdir: /home/matt/courses/software_engineering_best_practices
plugins: nbval-0.9.3
collected 4 items                                          

arrays.py::arrays.add_arrays PASSED                  [ 25%]
test_arrays.py::test_add_arrays[a0-b0-expect0] PASSED [ 50%]
test_arrays.py::test_add_arrays[a1-b1-expect1] PASSED [ 75%]
test_arrays.py::test_add_arrays_error PASSED         [100%]

===================== warnings summary =====================
venv/lib64/python3.7/site-packages/jinja2/utils.py:485
  /home/matt/courses/software_engineering_best_practices/venv/lib64/python3.7/site-packages/jinja2/utils.py:485: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
    

We see here the test `arrays.py::arrays.add_arrays` which has passed. Ignore the warning about deprecation, this is from a third-party module which is leaking through.

... doctests are really useful...

### Exercise

See what happens when you break your doctest and run `pytest` again.